In [7]:
from get_data import get_data
from dataset_sales import getDataset
import sales_prediction as sp
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os
import sys
import time

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\Embedding\\images_h.pt'
desc_path= 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\Embedding\\descrizioni.pt'

data_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\sales_week.csv'
data_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\sales_month.csv'
data_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\sales_season.csv'

noneg_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_week.csv'
noneg_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_month.csv'
noneg_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_season.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")

data, references, images, descriptions, target = get_data(data_week, desc_path, img_path)

modello = sp.create_model(len(data.columns)).to(device)
batch_size = 256
epochs = 100
criterion = nn.MSELoss()
early_stop = sp.EarlyStopping(5, 0.001)

optimizer = AdamW(
    [
        {"params": modello.image.parameters(), "lr": 1e-4},
        {"params": modello.tabular.parameters()},
        {"params": modello.description.parameters(), "lr": 1e-4},
        {"params": modello.final.parameters()}
    ],
    lr=1e-3,
)

train, val = getDataset(references, data, images, descriptions, target, batch_size, 1)

folder = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\week_negozi_huge'
if not os.path.isdir(folder):
    os.makedirs(folder)

min_mse = sys.maxsize
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    sp.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2, bias = sp.validation_loop(val, modello, criterion, device)
    
    print(f"\nValidation Error: \n Avg MSE: {mse:>8f} \n Avg MAE: {mae:>8f} \n R2: {r2:>8f}\n" + 
          f" Bias: {bias:>8f}\n")
    
    if (min_mse > mse):
        model_sc = torch.jit.script(modello)
        time.sleep(1)
        model_sc.save(f'{folder}\\weights.pt')
        min_mse = mse
        print(f'MINIMO MSE AGGIORNATO: {min_mse}\n\n')
        
    early_stp = early_stop(mse)
    
    if early_stp:
        print('Early Stop attivato.\n')
        break
    
print("Fatto!")

Sto usando cuda


c:\Users\GRVRLD00P\Documents\Progetto ORS\Progetto ORS\ProgettoORS\sales_prediction\get_data.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  descrizioni = torch.load(des

Epoch 1
-------------------------------
MSE: 4.686187  [  256/1391214]
MSE: 1.728581  [25856/1391214]
MSE: 1.438856  [51456/1391214]
MSE: 1.365910  [77056/1391214]
MSE: 1.737289  [102656/1391214]
MSE: 1.246668  [128256/1391214]
MSE: 1.095936  [153856/1391214]
MSE: 1.030962  [179456/1391214]
MSE: 1.346060  [205056/1391214]
MSE: 0.957845  [230656/1391214]
MSE: 1.532445  [256256/1391214]
MSE: 1.142364  [281856/1391214]
MSE: 0.896519  [307456/1391214]
MSE: 0.916048  [333056/1391214]
MSE: 1.472392  [358656/1391214]
MSE: 1.002625  [384256/1391214]
MSE: 1.220505  [409856/1391214]
MSE: 1.698076  [435456/1391214]
MSE: 1.511549  [461056/1391214]
MSE: 1.110321  [486656/1391214]
MSE: 1.041085  [512256/1391214]
MSE: 1.583815  [537856/1391214]
MSE: 1.563447  [563456/1391214]
MSE: 1.088116  [589056/1391214]
MSE: 1.113215  [614656/1391214]
MSE: 1.810197  [640256/1391214]
MSE: 0.924489  [665856/1391214]
MSE: 1.350616  [691456/1391214]
MSE: 1.157656  [717056/1391214]
MSE: 0.797726  [742656/1391214]
MSE: